In [1]:
import os

# Lấy thư mục hiện tại
current_directory = os.getcwd()
print("Current Directory:", current_directory)


Current Directory: /Users/minhquang/Documents/Learning/AI/DeepLearning/Project/HandwritterDigitClassifier


In [2]:
from keras.models import load_model

# Đường dẫn đến mô hình đã lưu
model_load_path = '/Users/minhquang/Documents/Learning/AI/DeepLearning/Project/HandwritterDigitClassifier/pre-train.keras'
# Tải lại mô hình
model = load_model(model_load_path)

print("Model has been loaded.")


Model has been loaded.


In [11]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipycanvas import Canvas

# Canvas
canvas = Canvas(width=280, height=280)
canvas.stroke_style = 'white'
canvas.line_width = 15
canvas.fill_style = 'black'
canvas.sync_image_data = True

# Biến để theo dõi xem chuột có đang nhấn hay không
is_drawing = False

# Khi nhấn chuột, bắt đầu vẽ
def on_mouse_down(x, y):
    global is_drawing
    is_drawing = True
    canvas.begin_path()
    canvas.move_to(x, y)

# Khi nhả chuột, kết thúc vẽ
def on_mouse_up(x, y):
    global is_drawing
    is_drawing = False

# Khi di chuyển chuột, nếu đang vẽ thì vẽ tiếp
def on_mouse_move(x, y):
    if is_drawing:
        canvas.line_to(x, y)
        canvas.stroke()

# Hàm xử lý và dự đoán số khi người dùng nhấn nút
def predict_digit(change=None):
    # Get image data from canvas
    image_data = np.array(canvas.get_image_data())

    # Convert to grayscale
    image_data_gray = np.mean(image_data[:, :, :3], axis=2)  # Grayscale

    # Display the drawn image
    plt.imshow(image_data, cmap='gray')
    plt.title("Drawn Image")
    plt.axis('off')
    plt.show()

    # Resize to 28x28 pixels for the MNIST model
    image_data = Image.fromarray(image_data_gray)
    image_data = image_data.resize((28, 28), Image.Resampling.LANCZOS)
    image_data = np.array(image_data)

    # Normalize and reshape the image for the model
    image_data = image_data / 255.0  
    image_data = np.expand_dims(image_data.flatten(), axis=0)  

    # Show the processed image
    # plt.imshow(image_data.reshape(28, 28), cmap='gray')
    # plt.title("Processed Image for Prediction")
    # plt.axis('off')
    # plt.show()

    # Predict using the model
    prediction = model.predict(image_data)[0]
    print("Prediction:", prediction)
    predicted_digit = np.argmax(prediction)   
    print("Predicted_digit:", predicted_digit)

    numbers = list(range(10))  # Digits 0 through 9
    plt.bar(numbers, prediction, color='blue', alpha=0.7)
    plt.title("Model Prediction Probabilities")
    plt.xlabel("Digits")
    plt.ylabel("Probability")
    plt.xticks(numbers)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Display prediction results
    print(f"Predicted Digit: {predicted_digit}")

    # Clear and reset the canvas
    canvas.clear()
    canvas.fill_rect(0, 0, canvas.width, canvas.height)  # Reset to black background


# Button to predict
predict_button = widgets.Button(description="Dự đoán Số")
predict_button.on_click(predict_digit)

# Assign event for mouse controll
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_up(on_mouse_up)
canvas.on_mouse_move(on_mouse_move)

# Display canvas and button
display(canvas, predict_button)


Canvas(height=280, sync_image_data=True, width=280)

Button(description='Dự đoán Số', style=ButtonStyle())